In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report, accuracy_score

In [4]:
df = pd.read_csv("dataset/clean_df.csv")

In [5]:
df.columns

Index(['Unnamed: 0', 'FIRST_SWITCHED', 'FLOW_DURATION_MILLISECONDS', 'IN_PKTS',
       'L4_DST_PORT', 'L4_SRC_PORT', 'LAST_SWITCHED', 'OUT_BYTES', 'OUT_PKTS',
       'SRC_TO_DST_SECOND_BYTES', 'TCP_FLAGS', 'TCP_WIN_MAX_OUT',
       'TCP_WIN_MSS_IN', 'LABEL', 'PROTOCOL_MAP_icmp', 'PROTOCOL_MAP_tcp',
       'PROTOCOL_MAP_udp', 'L7_PROTO_NAME_AJP', 'L7_PROTO_NAME_AMAZON',
       'L7_PROTO_NAME_APPLE', 'L7_PROTO_NAME_BGP', 'L7_PROTO_NAME_BITTORRENT',
       'L7_PROTO_NAME_CISCOVPN', 'L7_PROTO_NAME_CITRIX',
       'L7_PROTO_NAME_CLOUDFLARE', 'L7_PROTO_NAME_COAP', 'L7_PROTO_NAME_CORBA',
       'L7_PROTO_NAME_DCE_RPC', 'L7_PROTO_NAME_DNP3', 'L7_PROTO_NAME_DNS',
       'L7_PROTO_NAME_FACEBOOK', 'L7_PROTO_NAME_FTP_CONTROL',
       'L7_PROTO_NAME_FTP_DATA', 'L7_PROTO_NAME_GIT', 'L7_PROTO_NAME_GOOGLE',
       'L7_PROTO_NAME_H323', 'L7_PROTO_NAME_HTTP', 'L7_PROTO_NAME_HTTP_PROXY',
       'L7_PROTO_NAME_IAX', 'L7_PROTO_NAME_ICMP', 'L7_PROTO_NAME_IEC60870',
       'L7_PROTO_NAME_IMAP', 'L7_PROTO_NAM

In [6]:
len(df.columns)

97

In [7]:
# Drop the column unnamed
df = df.drop(["Unnamed: 0"], axis = 1)
len(df.columns)

96

In [8]:
df.describe()

,FIRST_SWITCHED,FLOW_DURATION_MILLISECONDS,IN_PKTS,L4_DST_PORT,L4_SRC_PORT,LAST_SWITCHED,OUT_BYTES,OUT_PKTS,SRC_TO_DST_SECOND_BYTES,TCP_FLAGS,...,L7_PROTO_NAME_TLS,L7_PROTO_NAME_UBNTAC2,L7_PROTO_NAME_UNKNOWN,L7_PROTO_NAME_VIBER,L7_PROTO_NAME_VNC,L7_PROTO_NAME_WHATSAPPFILES,L7_PROTO_NAME_WSD,L7_PROTO_NAME_XBOX,L7_PROTO_NAME_XDMCP,L7_PROTO_NAME_ZABBIX
count,9.767300e+04,97673.000000,97673.000000,97673.000000,97673.000000,9.767300e+04,97673.000000,97673.000000,97673.000000,97673.000000,...,97673.000000,97673.00000,97673.000000,97673.000000,97673.000000,97673.00000,97673.000000,97673.000000,97673.000000,97673.000000
mean,1.617767e+09,34460.110727,11.383504,11227.300482,42867.620100,1.617767e+09,4.776386,12.397889,5.600985,19.500712,...,0.064849,0.00001,0.317754,0.000031,0.000072,0.00001,0.000020,0.000031,0.000020,0.000020
std,7.146678e+05,42458.042828,202.795456,19192.477749,15615.596848,7.146899e+05,2.294412,262.955155,1.523983,10.367545,...,0.246261,0.00320,0.465606,0.005542,0.008465,0.00320,0.004525,0.005542,0.004525,0.004525
min,1.616660e+09,0.000000,1.000000,0.000000,0.000000,1.616660e+09,0.000000,0.000000,3.332205,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,1.616673e+09,0.000000,1.000000,80.000000,37424.000000,1.616673e+09,3.688879,1.000000,3.784190,19.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,1.618227e+09,401.000000,4.000000,80.000000,49214.000000,1.618227e+09,5.370638,3.000000,5.407172,24.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,1.618239e+09,60273.000000,11.000000,15086.000000,51229.000000,1.618239e+09,6.269096,10.000000,6.967909,27.000000,...,0.000000,0.00000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
max,1.618262e+09,119995.000000,32708.000000,65535.000000,65535.000000,1.618262e+09,17.939323,42335.000000,17.047960,223.000000,...,1.000000,1.00000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000


In [21]:
def scale_x(data):
    min_max_scaler = MinMaxScaler()
    return min_max_scaler.fit_transform(data)
    

In [25]:
def store_df(name, X, y):
    compression_opts = dict(method='zip',
                            archive_name=name+'_X.csv')  
    X.to_csv(name+'_X.zip', index=True,
              compression=compression_opts)  
    compression_opts = dict(method='zip',
                            archive_name=name+'_y.csv')  
    y.to_csv(name+'_y.zip', index=True,
              compression=compression_opts)  

In [22]:
random_forest_df = df.copy()

# Get representative sampling

We will get 100k sampling form each label right now each one has ~2M so we will get 5% of each one

In [ ]:
len(sample_df)

### Train, Test, Validation Split

In [ ]:
y = sample_df["LABEL"]
X = sample_df.drop(["LABEL"], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_beta, X_test, y_train_beta, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
validation_rate = 15/70
X_train, X_validation, y_train, y_validation = train_test_split(X_train_beta, y_train_beta, test_size=validation_rate, random_state=42)

In [112]:
# columns_to_change = [
#     "FIRST_SWITCHED", "FLOW_DURATION_MILLISECONDS", "IN_PKTS", "L4_DST_PORT", "L4_SRC_PORT", "LAST_SWITCHED",
#     "OUT_BYTES", "OUT_PKTS", "SRC_TO_DST_SECOND_BYTES"
# ]
# X_validation[columns_to_change] = scaler.fit_transform(X_validation[columns_to_change])
# X_train[columns_to_change] = scaler.fit_transform(X_train[columns_to_change])
# X_test[columns_to_change] = scaler.fit_transform(X_test[columns_to_change])

In [113]:
# store_df("test_values", X_test, y_test)
# store_df("train_values", X_train, y_train)
# store_df("validate_values", X_validation, y_validation)

## Random Forest Model

In [114]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

In [115]:
# Entrenamos el modelo
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=42)

In [116]:
# Realizamos predicciones con las metricas de validación
print("We predict validation")
predictions = rf.predict(X_validation)

We predict validation


#### Validación

In [117]:
# Get the results
validation_confusion_matrix = confusion_matrix(y_validation, predictions)
validation_recall_score = recall_score(y_validation, predictions, average=None)
validation_precision_score = precision_score(y_validation, predictions, average=None)
validation_f1_score = f1_score(y_validation, predictions, average=None)
print("Matrix de confusión: ", validation_confusion_matrix)
target_names =[    "Normal flow",
    "SYN Scan - aggressive",
    "Denial of Service R-U-Dead-Yet",
    "Denial of Service Slowloris"]
print(classification_report(y_validation, predictions, target_names = target_names))
print("recall_score: ",recall_score)
print("precision_score: ", precision_score)
print("f1_score: ", f1_score)
print("accuracy: ", accuracy_score(y_validation, predictions))

Matrix de confusión:  [[3805    0    0    3]
 [2759 1018    0    0]
 [   0    0 3314    0]
 [   3    0    0 1300]]
                                precision    recall  f1-score   support

                   Normal flow       0.58      1.00      0.73      3808
         SYN Scan - aggressive       1.00      0.27      0.42      3777
Denial of Service R-U-Dead-Yet       1.00      1.00      1.00      3314
   Denial of Service Slowloris       1.00      1.00      1.00      1303

                      accuracy                           0.77     12202
                     macro avg       0.89      0.82      0.79     12202
                  weighted avg       0.87      0.77      0.74     12202

recall_score:  <function recall_score at 0x152473040>
precision_score:  <function precision_score at 0x152468f70>
f1_score:  <function f1_score at 0x152468c10>
accuracy:  0.7733978036387478
